In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.linalg import null_space
from IPython.display import clear_output
import random
import warnings
warnings.filterwarnings('ignore')

In [3]:
X_small = pd.read_csv("../Data/BreastTissue.csv").drop(['Case #'],axis = 1).drop(['Class'],axis=1).to_numpy()
y_small = pd.read_csv("../Data/BreastTissue.csv").drop(['Case #'],axis = 1)['Class'].to_numpy()

y_small = preprocessing.LabelEncoder().fit_transform(y_small)

In [4]:
def reg_PCA(X, k = "all"): 
    """
    function takes an n x p feature matrix
    returns two arrays:
    - array with percentage of explained variance in first k principal directions (k_comp x 1)
    - array with principal directions (k_comp x p)
    """
    X = StandardScaler().fit_transform(X)
    if k == "all": k = min(X.shape[0],X.shape[1])
    pca = PCA(n_components = k)
    pca.fit(X)
    PEVs = pca.explained_variance_ratio_
    prin_comp = pca.components_
    EVs = pca.explained_variance_
    
    return PEVs, prin_comp, EVs

In [ ]:
def elasticNet_obj(X, v, reg_param, reg_param1):
    return

def elasticNet_grad(sigma, v, reg_param, reg_param1):
    return

In [ ]:
def elasticNetGradDesc(A_i, X, reg_param, reg_param1,
                       x0 = 'default', alpha = 1e-3, 
                       max_iter = 1000, crit = 1e-10):
    '''
    function takes
    - X: n x p dataset
    - A: p x n matrix
    
    function returns
    - B: p x n matrix
    '''
    # Configure the parameters
    iters, delta = 1, 1
    
    # Initialize the algorithm
    if x0 == 'default': x0 = np.ones(shape = (num_feat,1))
    elif x0 == 'random': x0 = np.random.rand(num_feat,1)
    else: pass
    v = x0/np.linalg.norm(x0)
    
    # Calculate initial value of objective function
    obj_cache = elasticNet_obj(X, v, reg_param, reg_param1)
    
     # Projected gradient descent
    # Stopping criteria:
    # (1) Maximum iterations reached
    # (2) Change in objective function negligible
    while iters < max_iter and delta > crit:
        
        # Update the vector
        v_new = v - alpha*elasticNet_grad(sigma, v, reg_param, reg_param1)
        
        # Project loading vector back onto feasible set 
        # (vectors of l2 norm of 1 that are orthogonal to all other pcs)
        v_proj = V @ v_new
        v_proj = v_proj/(np.linalg.norm(v_proj))
        
        # Use the projected loading veactor to retrieve the value of the
        # objective function
        updated_obj = elasticNet_obj(X, v_proj, reg_param, reg_param1)

        # Calculate the difference in value of the objective function
        delta = updated_obj - obj_cache
        
        # Update vector and number of iterations
        v, iters, obj_cache = v_proj, iters + 1, udated_obj
        
    return v

In [ ]:
def SVDProblem(B, X):
    '''
    function takes
    - X: n x p dataset
    - B: p x n matrix
    
    function returns
    - A: p x n matrix
    '''
    U, D, V = np.linalg.svd(X.T @ X @ B)
    return U @ V.T

In [ ]:
def SPCA(X, reg_param, reg_param1, 
         x0 = 'default', alpha = 1e-3, 
         max_iter = 1000, crit = 1e-10):
    '''
    function takes
    - X: n x p dataset
    - reg_param: regularization parameter (positive float)
    - x0: Initial value of the vector
    - alpha: Step size of gradient descent (<1 for convergence)
    - max_iters: max number of steps (positive integer)
    - crit: critical stopping value for the gradient descent algorithm
    
    function returns
    - array with principal components in its columns (n x p)
    '''
    # (1) Let A start at V[,1:k] the loadings of the first 
    # k ordinary principal components
    A = reg_PCA(X)[1].T
    B = np.zeros_like(A)
    
    iters, delta = 1, 1
    while iters < max_iter and delta > crit:
        # (2) Given a fixed A = [alpha_1, ..., alpha_k], solve the elastic
        # net problem for j = 1, 2, ..., k
        for i in range(len(A)):
            B[i] = elasticNetGradDesc(A[i], X, reg_param, reg_param1, 
                                      x0, alpha, max_iter, crit)
        
        # (3) For a fixed B = [beta_1, ..., beta_k], compute the SVD of 
        # X^TXB = UDV^T then update A = UV^T
        A = SVDProblem(B, X)
        
        # (4) Repeat steps (2) and (3) until convergence
        delta, A_old = np.linalg.norm(A - A_old), A
    
    # (5) Normalize the altered principal components
    return (A/np.linalg.norm(A, axis = 1)).T